In [1]:
# 서울버스정류장
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from datetime import datetime
from ServiceKey import public_serviceKey, mySQL_Connect

mydb = mySQL_Connect()

cursor = mydb.cursor(prepared=True)

# 2021.09.21일 수집 완료
query = ("select seoulsttsName from public_seoulbusstts ps where 7023 <= seq and seq <= 8023")

cursor.execute(query)

location_list = []

for i in cursor:
    location_list.append(i)
    
now = datetime.now()
nowDate = now.strftime('%Y%m%d')

cnt = 1
for i in location_list:
    seoulsttsName = i
    
    characters = "(',')"

    join_data = ''.join( x for x in seoulsttsName if x not in characters)
    print("조회날짜 : {} | 순번 : {} | start {} public_seoulbusstts".format(nowDate, cnt, join_data))
    url = 'http://ws.bus.go.kr/api/rest/stationinfo/getStationByName'
    queryParams = '?' + urlencode({
        quote_plus('serviceKey') : public_serviceKey(),
        quote_plus('stSrch') : join_data
    })

    response = requests.get(url + queryParams).text.encode('utf-8')
    xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

    rows = xmlobj.findAll('itemList')
    resultCode = xmlobj.findAll('headerCd')

    if resultCode[0].text == '7':
        print("{} 번째부터 조회수 제한이 걸렸습니다.".format(seoulsttsName))
        mydb.close()
        break
    cnt += 1
    for j in rows:
        item = j
        arsId = item.findAll('arsId')
        stId = item.findAll('stId')
        stNm = item.findAll('stNm')
        lat = item.findAll('tmY')
        lng = item.findAll('tmX')
        
        try:
            sql = "INSERT INTO public_seoulbusarsid_info (arsId, stId, stNm, lat, lng) VALUES (?,?,?,?,?)"
            val = (arsId[0].text, stId[0].text, stNm[0].text, lat[0].text, lng[0].text)
        except IndexError:
            print(item)
            sql = "INSERT INTO public_seoulbusarsid_info (arsId, stId, stNm, lat, lng) VALUES (?,?,?,?,?)"
            val = (arsId[0].text, stId[0].text, stNm[0].text, lat[0].text, lng[0].text)
            
        cursor.execute(sql, val)
        mydb.commit()

조회날짜 : 20210921 | 순번 : 1 | start 동일하이빌아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 2 | start 팔복부동산.목동교회 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 3 | start 신정현대아파트.서부트럭터미널 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 4 | start 목동문화체육센터.건영아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 5 | start 한성교회.금옥여고.백암고 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 6 | start 우신웨스트빌.목동보건지소 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 7 | start 대림.수정아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 8 | start 벽산.대주아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 9 | start 성원.극동아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 10 | start 목동센트럴2단지 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 11 | start 목동센트럴1단지 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 12 | start 농협.영도초등학교 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 13 | start 학마을2단지아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 14 | start 한신청구아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 15 | start 학마을2단지아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 16 | start 신정1

조회날짜 : 20210921 | 순번 : 131 | start 강서면허시험장.메이필드호텔 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 132 | start 강서면허시험장.메이필드호텔 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 133 | start 강서농수산물도매시장 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 134 | start 영인운수.차고지 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 135 | start 대한항공.메이필드호텔 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 136 | start 공항동송정중학교 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 137 | start 공항동송정중학교 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 138 | start 국립항공박물관.공항동천주교회 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 139 | start 국립항공박물관.공항동천주교회 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 140 | start 공항동지하도입구 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 141 | start 영인운수.차고지 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 142 | start 방화중학교 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 143 | start 방화2동주민센터 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 144 | start 방화동도시개발11단지 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 145 | start 개화2치안센터 public_seoulbusstts
조회날짜 : 202

조회날짜 : 20210921 | 순번 : 258 | start 방화동동부센트레빌아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 259 | start 방화동동부센트레빌아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 260 | start 방화사거리 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 261 | start 방화사거리 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 262 | start 개화산역 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 263 | start 개화산역 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 264 | start 등촌3동천주교회 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 265 | start 등촌3동천주교회 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 266 | start 등촌3동5단지아파트.등촌고등학교 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 267 | start 등촌3동5단지아파트.등촌고등학교 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 268 | start 등촌3동주공4단지아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 269 | start 등촌동부영아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 270 | start 등촌3동주공10단지아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 271 | start 등촌3동대림아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 272 | start 마곡13단지힐스테이트아파트 public_seoulbusstts
조회날짜 : 20210

조회날짜 : 20210921 | 순번 : 388 | start 신광명마을 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 389 | start 수명중고등학교 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 390 | start 마곡.수명산파크4단지 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 391 | start 방화중학교 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 392 | start 한마음아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 393 | start 우성아파트앞 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 394 | start 우성아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 395 | start 봉제산정상 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 396 | start 생태공원 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 397 | start NC백화점앞 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 398 | start 염창역1번출구.강서평생학습관 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 399 | start 명지해드는터아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 400 | start 하이웨이주유소 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 401 | start 서광아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 402 | start 마곡엠밸리7단지앞 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 403 | start 대림아파트 public_seoulbu

조회날짜 : 20210921 | 순번 : 522 | start 동아1.2차아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 523 | start 약수터 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 524 | start 동아1.2차아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 525 | start 화곡역 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 526 | start 마곡우림필유아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 527 | start 마곡우림필유아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 528 | start 서울신정초등학교 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 529 | start 대영마트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 530 | start 롯데R&D센터 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 531 | start 마곡엠밸리7단지.712동 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 532 | start 신방화역4번출구.마곡중학교 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 533 | start 신방화역7번출구 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 534 | start 마곡엠밸리4단지 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 535 | start 마곡엠밸리5단지 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 536 | start 화곡중앙골목시장 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 537 | start 송화시장 

조회날짜 : 20210921 | 순번 : 653 | start 대한항공훈련센터 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 654 | start 김포공항화물청사 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 655 | start 김포공항세관.국립항공박물관 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 656 | start 강원슈퍼 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 657 | start 대한항공빌딩 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 658 | start 대한항공 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 659 | start 김포공항국제선 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 660 | start 신도림역 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 661 | start 신도림역 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 662 | start 신도림동.구로역 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 663 | start 신도림동.구로역 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 664 | start 구일역.중앙유통상가.롯데마트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 665 | start 구일역.제일제당.롯데마트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 666 | start 동양미래대학.구로성심병원 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 667 | start 동양미래대학.구로성심병원 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 668 | st

조회날짜 : 20210921 | 순번 : 781 | start 세곡초등학교 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 782 | start 세곡초등학교 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 783 | start 개봉푸르지오아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 784 | start 개봉푸르지오아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 785 | start 고척근린시장 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 786 | start 용두연립 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 787 | start 우리유엔미아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 788 | start 쌍용구로정비사업소 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 789 | start 고척파크푸르지오아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 790 | start 고척시장 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 791 | start 고척시장 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 792 | start 고척중학교 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 793 | start 고척중학교 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 794 | start 구로디지털단지역 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 795 | start 삼부르네상스.효성아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 796 | start 구일우성아파트 public_seou

조회날짜 : 20210921 | 순번 : 912 | start 개봉역.림괄아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 913 | start 덕의초등학교.고척동성당 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 914 | start 이마트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 915 | start 구로고대병원정문 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 916 | start 대림5차아파트.신도림주민센터 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 917 | start 세곡초등학교 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 918 | start 오류2동주민센터.오류남초교 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 919 | start 새마을금고사거리 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 920 | start 삼환로노즈빌아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 921 | start 낙원할인마트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 922 | start 오류고등학교.수궁동복지회관 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 923 | start 경북슈퍼마켓 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 924 | start 대림5차아파트702동 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 925 | start 남부교회 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 926 | start 서울가든아파트 public_seoulbusstts
조회날짜 : 20210921 | 순번 : 927 |